In [1]:
import pandas as pd
import numpy as np
import json, csv
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from keras.models import load_model


Using TensorFlow backend.


In [2]:
with open('mappings.json') as f: MAPPINGS = json.load(f)
FLAG          = MAPPINGS['flag']
SERVICE       = MAPPINGS['service']
PROTOCOL_TYPE = MAPPINGS['protocol_type']

In [3]:
train_data=pd.read_csv('kdd_train.csv')
train_data['protocol_type'] = train_data['protocol_type'].map(PROTOCOL_TYPE)
train_data['flag']          = train_data['flag'].map(FLAG)
train_data['service']       = train_data['service'].map(SERVICE)

In [4]:
train_data['labels'].unique()

array(['normal', 'neptune', 'warezclient', 'ipsweep', 'portsweep',
       'teardrop', 'nmap', 'satan', 'smurf', 'pod', 'back',
       'guess_passwd', 'ftp_write', 'multihop', 'rootkit',
       'buffer_overflow', 'imap', 'warezmaster', 'phf', 'land',
       'loadmodule', 'spy', 'perl'], dtype=object)

In [5]:
DOS_lis=['back','land','neptune','pod','smurf','teardrop']
Probe_lis=['ipsweep','portsweep','nmap','satan']
R2l_lis=['guess_passwd','ftp_write','imap','phf','multihop','warezmaster']
U2R_lis=['buffer_overflow','loadmodule','rootkit','perl']
other_lis=['warezclient','spy']

In [6]:
d = train_data['labels'].isin(DOS_lis)
p=train_data['labels'].isin(Probe_lis)
r=train_data['labels'].isin(R2l_lis)
u=train_data['labels'].isin(U2R_lis)
o=train_data['labels'].isin(other_lis)

In [7]:
train_data['labels']=train_data['labels'].mask(d,'DOS')
train_data['labels']=train_data['labels'].mask(p,'Probe')
train_data['labels']=train_data['labels'].mask(r,'R2L')
train_data['labels']=train_data['labels'].mask(u,'U2R')
train_data['labels']=train_data['labels'].mask(o,'Other')

In [8]:
train_data['labels'].unique()

array(['normal', 'DOS', 'Other', 'Probe', 'R2L', 'U2R'], dtype=object)

In [10]:
features = train_data.columns.tolist()[:-1]
# X_DOS=DOS_train_data[features]
# X_Other=Other_train_data[features]
# X_Probe=Probe_train_data[features]
# X_R2L=R2L_train_data[features]
# X_U2R=u2R_train_data[features]



In [12]:
X_train=train_data[features]
X_normalized, norms = normalize(X_train, norm='l2', return_norm=True)
pca= PCA(n_components=5) 
pca.fit(X_normalized)

PCA(n_components=5)

In [14]:
from numpy.random import seed
#from tensorflow import set_random_seed
from keras.layers import Input,Dropout
from keras.layers.core import Dense
from keras.models import Model,Sequential,load_model
from keras import regularizers
from keras.models import model_from_json

In [27]:
import keras
anomaly_train_data=train_data.loc[train_data['labels']!='normal']
#anomaly_train_data=train_data.loc[train_data['labels']!='Other']
X_train_anomaly=anomaly_train_data[features]
X_normalized_anomaly, norms = normalize(X_train_anomaly, norm='l2', return_norm=True)
X_reduced_anomaly=pca.transform(X_normalized_anomaly)
Y=anomaly_train_data['labels']
Y_train=pd.get_dummies(Y).values
print(Y_train)

[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 ...
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]]


In [28]:
print(Y.unique())

['DOS' 'Other' 'Probe' 'R2L' 'U2R']


In [29]:
seed(10)
act_func='elu'
model=Sequential()
model.add(Dense(8,activation=act_func,kernel_initializer='glorot_uniform',kernel_regularizer=regularizers.l2(0.0),input_shape=(X_reduced_anomaly.shape[1],)))
model.add(Dense(8,activation=act_func,kernel_initializer='glorot_uniform'))
model.add(Dense(5,activation='softmax',kernel_initializer='glorot_uniform'))
model.compile(loss='categorical_crossentropy',optimizer='adam')
NUM_EPOCHS=30
BATCH_SIZE=10

In [30]:
history=model.fit(np.array(X_reduced_anomaly),Y_train,batch_size=BATCH_SIZE,epochs=50,validation_split=0.05,verbose=1)

Train on 55698 samples, validate on 2932 samples
Epoch 1/50
55698/55698 [==============================] - 8s 145us/step - loss: 0.3434 - val_loss: 0.2553
Epoch 2/50
55698/55698 [==============================] - 8s 141us/step - loss: 0.2414 - val_loss: 0.2195
Epoch 3/50
55698/55698 [==============================] - 8s 141us/step - loss: 0.2194 - val_loss: 0.2052
Epoch 4/50
55698/55698 [==============================] - 9s 163us/step - loss: 0.2093 - val_loss: 0.1967
Epoch 5/50
55698/55698 [==============================] - 9s 163us/step - loss: 0.1993 - val_loss: 0.1882
Epoch 6/50
55698/55698 [==============================] - 8s 152us/step - loss: 0.1909 - val_loss: 0.1831
Epoch 7/50
55698/55698 [==============================] - 8s 143us/step - loss: 0.1843 - val_loss: 0.1736
Epoch 8/50
55698/55698 [==============================] - 9s 153us/step - loss: 0.1717 - val_loss: 0.1551
Epoch 9/50
55698/55698 [==============================] - 8s 141us/step - loss: 0.1578 - val_loss: 0.15

In [31]:
model.save('network2.h5')

In [46]:
y_pred=model.predict(X_reduced_anomaly)

In [47]:
y_train_class=np.argmax(Y_train,axis=1)

In [48]:
y_pred_class=np.argmax(y_pred,axis=1)

In [49]:
from sklearn.metrics import classification_report,confusion_matrix

In [50]:
print(classification_report(y_train_class,y_pred_class))
print(confusion_matrix(y_train_class,y_pred_class))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97     45927
           1       0.81      0.94      0.87       892
           2       0.92      0.88      0.90     11656
           3       0.64      0.72      0.68       103
           4       0.64      0.13      0.22        52

    accuracy                           0.96     58630
   macro avg       0.80      0.73      0.73     58630
weighted avg       0.96      0.96      0.96     58630

[[44984   127   810     6     0]
 [    5   835    51     1     0]
 [ 1363    39 10242    12     0]
 [    7     9     9    74     4]
 [    2    16     5    22     7]]


In [51]:
test_data=pd.read_csv('kdd_test.csv')
test_data['protocol_type'] = test_data['protocol_type'].map(PROTOCOL_TYPE)
test_data['flag']          = test_data['flag'].map(FLAG)
test_data['service']       = test_data['service'].map(SERVICE)

In [52]:
DOS_lis=['back','land','neptune','pod','smurf','teardrop','mailbomb','processtable','udpstorm','apache2','worm']
Probe_lis=['ipsweep','portsweep','nmap','satan','nmap','mscan','saint']
R2l_lis=['guess_passwd','ftp_write','imap','phf','multihop','warezmaster','xlock','xsnoop','snmpguess','snmpgetattack','httptunnel','sendmail','named']
U2R_lis=['buffer_overflow','loadmodule','rootkit','perl','sqlattack','xterm','ps']
other_lis=['warezclient','spy']

In [53]:
d = test_data['labels'].isin(DOS_lis)
p=test_data['labels'].isin(Probe_lis)
r=test_data['labels'].isin(R2l_lis)
u=test_data['labels'].isin(U2R_lis)
o=test_data['labels'].isin(other_lis)

In [54]:
test_data['labels']=test_data['labels'].mask(d,'DOS')
test_data['labels']=test_data['labels'].mask(p,'Probe')
test_data['labels']=test_data['labels'].mask(r,'R2L')
test_data['labels']=test_data['labels'].mask(u,'U2R')
test_data['labels']=test_data['labels'].mask(o,'Other')

In [55]:
anomaly_test_data=test_data.loc[test_data['labels']!='normal']
X_test_anomaly=anomaly_test_data[features]
Xt_normalized_anomaly, norms = normalize(X_test_anomaly, norm='l2', return_norm=True)
Xt_reduced_anomaly=pca.transform(Xt_normalized_anomaly)
Yt=anomaly_test_data['labels']
Y_test=pd.get_dummies(Yt).values

In [56]:
print(Yt.unique())

['Probe' 'DOS' 'R2L' 'Other' 'U2R']


In [57]:
yt_pred=model.predict(Xt_reduced_anomaly)

In [58]:
y_test_class=np.argmax(Y_test,axis=1)
yt_pred_class=np.argmax(yt_pred,axis=1)

In [59]:
print(classification_report(y_test_class,yt_pred_class))
print(confusion_matrix(y_test_class,yt_pred_class))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      8095
           1       0.26      0.94      0.40       107
           2       0.64      0.81      0.71      2157
           3       0.52      0.03      0.05       902
           4       0.60      0.08      0.14        38

    accuracy                           0.84     11299
   macro avg       0.59      0.56      0.45     11299
weighted avg       0.84      0.84      0.82     11299

[[7587  169  338    1    0]
 [   2  101    4    0    0]
 [ 400    5 1748    4    0]
 [ 123  106  646   25    2]
 [   2   11    4   18    3]]
